# Mouse Facial Expressions - 로컬 학습

이 노트북은 로컬 Anaconda 환경에서 모델을 학습하기 위한 것입니다.

## 시작하기 전에:
1. Anaconda가 설치되어 있어야 합니다
2. 터미널에서 실행: `jupyter notebook`
3. 이 노트북을 열고 실행

## 예상 시간:
- CPU: 7-8시간 (밤새 실행 추천)
- 각 epoch: 약 40-50분

## Step 1: 환경 확인

In [32]:
import os
import sys

# 프로젝트 디렉토리로 이동
project_dir = '/Users/minakang/Desktop/mouse-facial-expressions-2023-main'
os.chdir(project_dir)

print(f"현재 디렉토리: {os.getcwd()}")
print(f"Python 버전: {sys.version}")

현재 디렉토리: /Users/minakang/Desktop/mouse-facial-expressions-2023-main
Python 버전: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]


In [33]:
# 필요한 패키지 확인
import torch
import lightning
import mlflow

print(f"PyTorch 버전: {torch.__version__}")
print(f"Lightning 버전: {lightning.__version__}")
print(f"MLflow 버전: {mlflow.__version__}")

# CPU/GPU 확인
print(f"\nCUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")
print("\n⚠️ 이 노트북은 CPU로 학습합니다 (안정성을 위해)")

PyTorch 버전: 2.9.1
Lightning 버전: 2.5.6
MLflow 버전: 3.6.0

CUDA available: False
MPS available: True

⚠️ 이 노트북은 CPU로 학습합니다 (안정성을 위해)


## Step 2: 데이터 확인

In [34]:
# 로컬 프레임 확인
!ls -lh data/local_frames/ | head -20

total 0
drwx------  199 minakang  staff   6.2K Nov 23 15:37 f1_rec0_acclimation
drwx------  195 minakang  staff   6.1K Nov 23 15:37 f1_rec1_preinjection
drwx------  197 minakang  staff   6.2K Nov 23 15:40 f1_rec2_1h-postinjection
drwx------  194 minakang  staff   6.1K Nov 23 15:41 f1_rec3_2h-postinjection
drwx------  201 minakang  staff   6.3K Nov 23 15:39 f1_rec4_4h-postinjection
drwx------  193 minakang  staff   6.0K Nov 23 15:37 f10_rec0_acclimation
drwx------  190 minakang  staff   5.9K Nov 23 15:36 f10_rec1_preinjection
drwx------  199 minakang  staff   6.2K Nov 23 15:36 f10_rec2_1h-postinjection
drwx------  201 minakang  staff   6.3K Nov 23 15:35 f10_rec3_2h-postinjection
drwx------  201 minakang  staff   6.3K Nov 23 15:34 f10_rec4_4h-postinjection
drwx------  203 minakang  staff   6.3K Nov 23 15:31 f11_rec0_acclimation
drwx------  200 minakang  staff   6.3K Nov 23 15:34 f11_rec1_preinjection
drwx------  202 minakang  staff   6.3K Nov 23 15:34 f11_rec2_1h-postinjection
drwx------

In [35]:
# 데이터셋 확인
!ls -lh data/processed/task-1.1/

total 12200
-rw-------  1 minakang  staff   1.5M Nov 23 09:19 dataset_df.pkl
-rw-------  1 minakang  staff   168K Nov 23 09:20 eval.pkl
-rw-------  1 minakang  staff   538K Nov 23 09:19 fold0.pkl
-rw-------  1 minakang  staff   538K Nov 23 09:19 fold1.pkl
-rw-------  1 minakang  staff   538K Nov 23 09:19 fold2.pkl
-rw-------  1 minakang  staff   538K Nov 23 09:19 fold3.pkl
-rw-------  1 minakang  staff   538K Nov 23 09:20 fold4.pkl
-rw-------  1 minakang  staff   538K Nov 23 09:20 fold5.pkl
-rw-------  1 minakang  staff   538K Nov 23 09:20 fold6.pkl
-rw-------  1 minakang  staff   538K Nov 23 09:20 fold7.pkl
-rw-------  1 minakang  staff   395B Nov 23 09:19 README.txt


In [36]:
# 프레임 개수 확인
!find data/local_frames -name "*.png" | wc -l

   32771


## Step 3: 환경 변수 확인

In [46]:
env_content = """MFE_RAW_VIDEO_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/raw
MFE_RAW_CSV_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/raw
MFE_VERSION=20230627
MFE_PROCESSED_VIDEO_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/processed
MFE_EXTRACTED_FRAMES_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/frames_backup
MFE_TASKS=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/processed
"""

with open('.env', 'w') as f:
    f.write(env_content)
print("✅ .env 업데이트 완료!")

✅ .env 업데이트 완료!


In [38]:
# .env 파일 확인
!cat .env

MFE_RAW_VIDEO_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/raw
MFE_RAW_CSV_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/raw
MFE_VERSION=20230627
MFE_PROCESSED_VIDEO_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/processed
MFE_EXTRACTED_FRAMES_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/local_frames
MFE_TASKS=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/processed


In [47]:
!python mouse_facial_expressions/data/make_datasets.py task1 --version "1.1" --frameset_size 1

2025-11-23 15:57:42,908 - __main__ - INFO - 
Task1 classification task using sets of 1 frames

Classes:
- 0: All animals at preinjection
- 1: LPS High dose at 4 hours

To balance the training datset (n=10000), classes are first sampled
randomly then videos are randomly sampled and finally frames.

The testing dataset (n=1000) only samples by video, preserving imbalances
in the dataset.

Split over 8 stratified kfolds grouped by mouse.

2025-11-23 15:57:42,908 - __main__ - INFO - Seeding 13641
2025-11-23 15:57:42,908 - __main__ - INFO - Loading treatment csv
Listing directories in /Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/frames_backup...
Traceback (most recent call last):
  File "/Users/minakang/Desktop/mouse-facial-expressions-2023-main/mouse_facial_expressions/data/make_datasets.py", line 406, in <module>
    main()
    ~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/click/core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
        

## Step 4: 학습 시작

### Option A: 전체 학습 (10 epochs, 7-8시간)

In [39]:
# 전체 학습 실행
!python mouse_facial_expressions/models/train_task1_baseline_model.py \
    --epochs 10 \
    --dataset_version "1.1"

2025-11-23 15:56:29,614 - __main__ - INFO - beginning model run train_task1_baseline_model.py
/opt/anaconda3/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/23 15:56:30 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Cmd('git') failed due to: exit code(1)
  cmdline: git version
  stderr: 'xcode-select: note: No developer tools were found, requesting install.
If developer tools are located at a non-default location on disk, use `sudo xcode-select --switch path/to/Xcode.app` to specify the Xcode that you wish to use for command line developer tools, and cancel the installation dialog.
See `man 

### Option B: 테스트 실행 (1 epoch, 40-50분)

In [40]:
# 1 epoch 테스트
!python mouse_facial_expressions/models/train_task1_baseline_model.py \
    --epochs 1 \
    --dataset_version "1.1"

2025-11-23 15:57:01,608 - __main__ - INFO - beginning model run train_task1_baseline_model.py
/opt/anaconda3/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/23 15:57:01 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Cmd('git') failed due to: exit code(1)
  cmdline: git version
  stderr: 'xcode-select: note: No developer tools were found, requesting install.
If developer tools are located at a non-default location on disk, use `sudo xcode-select --switch path/to/Xcode.app` to specify the Xcode that you wish to use for command line developer tools, and cancel the installation dialog.
See `man 

## Step 5: 학습 모니터링

학습이 진행되는 동안 다른 셀에서 진행 상황을 확인할 수 있습니다.

In [41]:
# 체크포인트 확인
!ls -lht models/checkpoints/ 2>/dev/null || echo "아직 체크포인트가 없습니다"

total 0


In [42]:
# 프로세스 확인
!ps aux | grep train_task1 | grep -v grep

## Step 6: 결과 확인

In [43]:
# 저장된 체크포인트 목록
import glob

checkpoints = glob.glob('models/checkpoints/*.ckpt')
if checkpoints:
    print("저장된 체크포인트:")
    for ckpt in sorted(checkpoints):
        size = os.path.getsize(ckpt) / (1024*1024)  # MB
        print(f"  {os.path.basename(ckpt)} ({size:.1f} MB)")
else:
    print("아직 체크포인트가 없습니다")

아직 체크포인트가 없습니다


In [44]:
# 최고 성능 모델 찾기
import re

best_acc = 0
best_ckpt = None

for ckpt in checkpoints:
    # 파일명에서 val_acc 추출
    match = re.search(r'val_acc=([0-9.]+)', ckpt)
    if match:
        acc = float(match.group(1))
        if acc > best_acc:
            best_acc = acc
            best_ckpt = ckpt

if best_ckpt:
    print(f"\n최고 성능 모델:")
    print(f"  파일: {os.path.basename(best_ckpt)}")
    print(f"  Validation Accuracy: {best_acc:.2%}")
else:
    print("체크포인트를 찾을 수 없습니다")

체크포인트를 찾을 수 없습니다


## Step 7: 체크포인트 로드 및 테스트 (선택사항)

In [45]:
# 체크포인트 로드 예시
if best_ckpt:
    checkpoint = torch.load(best_ckpt)
    print("체크포인트 정보:")
    print(f"  Epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"  Keys: {list(checkpoint.keys())}")

## 다음 단계: Chronic 데이터로 Fine-tuning

학습이 완료되면 `FINE_TUNING_GUIDE.md`를 참고하여 chronic 데이터로 fine-tuning하세요.

```python
# Fine-tuning 예시
!python mouse_facial_expressions/models/finetune_chronic.py \
    --checkpoint_path models/checkpoints/task1-fold0-epoch=9-val_acc=0.95.ckpt \
    --epochs 5 \
    --dataset_version "chronic_v1"
```

## 유용한 명령어

### 백그라운드로 실행하기

노트북을 닫아도 학습이 계속되도록 하려면 터미널에서:

```bash
cd /Users/minakang/Desktop/mouse-facial-expressions-2023-main
nohup python mouse_facial_expressions/models/train_task1_baseline_model.py \
    --epochs 10 --dataset_version "1.1" > training_log.txt 2>&1 &
```

### 진행 상황 확인

```bash
# 로그 확인
tail -f training_log.txt

# 프로세스 확인
ps aux | grep train_task1

# 체크포인트 확인
ls -lht models/checkpoints/
```